In [1]:
import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
class Classifier(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.quant = torch.ao.quantization.QuantStub()
        self.model = nn.Sequential(
            nn.Conv2d(1, 8, 3, stride=2),
            nn.ReLU(),
            nn.Conv2d(8, 8*8, 3, stride=2),
            nn.ReLU(),
            nn.AvgPool2d(3, 2),
            nn.Flatten(),
            nn.Dropout(),
            nn.Linear(256, 128), 
            nn.Dropout(),
            nn.Linear(128, 10) 
            # nn.Softmax()
        )
        self.dequant = torch.ao.quantization.DeQuantStub()

    def forward(self, x):
        x = self.quant(x)
        y = self.model(x)
        return self.dequant(y)
    

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X = X.to("cuda")
        y = y.to("cuda")
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [3]:
model = Classifier()

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
loss_fn = nn.CrossEntropyLoss()

In [4]:
model = model.to("cuda")
for i in range(10):
    train(train_dataloader, model, loss_fn, optimizer)
model = model.to("cpu")


loss: 2.298657  [   64/60000]
loss: 2.298033  [ 6464/60000]
loss: 2.306411  [12864/60000]
loss: 2.297748  [19264/60000]
loss: 2.309016  [25664/60000]
loss: 2.300261  [32064/60000]
loss: 2.311506  [38464/60000]
loss: 2.297639  [44864/60000]
loss: 2.302421  [51264/60000]
loss: 2.320634  [57664/60000]
loss: 2.314624  [   64/60000]
loss: 2.304060  [ 6464/60000]
loss: 2.293950  [12864/60000]
loss: 2.298303  [19264/60000]
loss: 2.297633  [25664/60000]
loss: 2.298316  [32064/60000]
loss: 2.308185  [38464/60000]
loss: 2.302151  [44864/60000]
loss: 2.301203  [51264/60000]
loss: 2.288205  [57664/60000]
loss: 2.304802  [   64/60000]
loss: 2.287807  [ 6464/60000]
loss: 2.309642  [12864/60000]
loss: 2.294869  [19264/60000]
loss: 2.299878  [25664/60000]
loss: 2.297906  [32064/60000]
loss: 2.298508  [38464/60000]
loss: 2.295811  [44864/60000]
loss: 2.291297  [51264/60000]
loss: 2.312205  [57664/60000]
loss: 2.305930  [   64/60000]
loss: 2.291296  [ 6464/60000]
loss: 2.291963  [12864/60000]
loss: 2.28

In [5]:
%%timeit
test(test_dataloader, model, loss_fn)

Test Error: 
 Accuracy: 34.0%, Avg loss: 1.996007 

Test Error: 
 Accuracy: 34.0%, Avg loss: 1.996007 

Test Error: 
 Accuracy: 34.0%, Avg loss: 1.996007 

Test Error: 
 Accuracy: 34.0%, Avg loss: 1.996007 

Test Error: 
 Accuracy: 34.0%, Avg loss: 1.996007 

Test Error: 
 Accuracy: 34.0%, Avg loss: 1.996007 

Test Error: 
 Accuracy: 34.0%, Avg loss: 1.996007 

Test Error: 
 Accuracy: 34.0%, Avg loss: 1.996007 

324 ms ± 1.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
model = model.to("cpu")

model.eval()
backend = "x86"
model.qconfig = torch.quantization.get_default_qconfig(backend)
torch.backends.quantized.engine = backend

model_static_quantized = torch.quantization.prepare(model, inplace=False)
model_static_quantized = torch.quantization.convert(model_static_quantized, inplace=False)

/home/arthur/miniconda3/lib/python3.11/site-packages/torch/ao/quantization/observer.py:221: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
/home/arthur/miniconda3/lib/python3.11/site-packages/torch/ao/quantization/observer.py:1289: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(


In [7]:
print(model_static_quantized)
for p in model_static_quantized.parameters():
    print(p)
    print(type(p))

Classifier(
  (quant): Quantize(scale=tensor([1.]), zero_point=tensor([0]), dtype=torch.quint8)
  (model): Sequential(
    (0): QuantizedConv2d(1, 8, kernel_size=(3, 3), stride=(2, 2), scale=1.0, zero_point=0)
    (1): ReLU()
    (2): QuantizedConv2d(8, 64, kernel_size=(3, 3), stride=(2, 2), scale=1.0, zero_point=0)
    (3): ReLU()
    (4): AvgPool2d(kernel_size=3, stride=2, padding=0)
    (5): Flatten(start_dim=1, end_dim=-1)
    (6): QuantizedDropout(p=0.5, inplace=False)
    (7): QuantizedLinear(in_features=256, out_features=128, scale=1.0, zero_point=0, qscheme=torch.per_channel_affine)
    (8): QuantizedDropout(p=0.5, inplace=False)
    (9): QuantizedLinear(in_features=128, out_features=10, scale=1.0, zero_point=0, qscheme=torch.per_channel_affine)
  )
  (dequant): DeQuantize()
)


In [8]:
%%timeit
test(test_dataloader, model_static_quantized, loss_fn)

Test Error: 
 Accuracy: 12.4%, Avg loss: 2.275030 

Test Error: 
 Accuracy: 12.4%, Avg loss: 2.275030 

Test Error: 
 Accuracy: 12.4%, Avg loss: 2.275030 

Test Error: 
 Accuracy: 12.4%, Avg loss: 2.275030 

Test Error: 
 Accuracy: 12.4%, Avg loss: 2.275030 

Test Error: 
 Accuracy: 12.4%, Avg loss: 2.275030 

Test Error: 
 Accuracy: 12.4%, Avg loss: 2.275030 

Test Error: 
 Accuracy: 12.4%, Avg loss: 2.275030 

317 ms ± 6.85 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
torch.save(model_static_quantized, "model_post_static_quantized.pt")